In [18]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [19]:
df=pd.read_csv("/workspaces/desktop-tutorial/mysample/anime.csv")

In [20]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [21]:
df.shape

(12294, 7)

In [22]:
df.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [23]:
df["rating"]=df["rating"].fillna(df["rating"].mean())
df["genre"]=df["genre"].fillna(df["genre"].mode()[0])
df["type"]=df["type"].fillna(df["type"].mode()[0])

In [24]:
df["rating"].describe()

count    12294.000000
mean         6.473902
std          1.017096
min          1.670000
25%          5.900000
50%          6.550000
75%          7.170000
max         10.000000
Name: rating, dtype: float64

In [25]:
df.isna().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [27]:
x=df.iloc[0]
x

anime_id                                   32281
name                              Kimi no Na wa.
genre       Drama, Romance, School, Supernatural
type                                       Movie
episodes                                       1
rating                                      9.37
members                                   200630
Name: 0, dtype: object

In [28]:
strlist=str(x['genre'])+", "+str(x['type'])
strlist=strlist.split(', ')
strlist=set(strlist)
strlist

{'Drama', 'Movie', 'Romance', 'School', 'Supernatural'}

In [29]:
def convert_into_set(x):
    strlist=str(x['genre'])+", "+str(x['type'])
    strlist=strlist.split(', ')
    strlist=set(strlist)
    return strlist

In [30]:
df['set']=df.apply(convert_into_set,axis=1)
df['set']

0            {Drama, School, Romance, Movie, Supernatural}
1        {Adventure, TV, Shounen, Drama, Action, Milita...
2        {TV, Shounen, Sci-Fi, Action, Parody, Samurai,...
3                                   {Sci-Fi, TV, Thriller}
4        {TV, Shounen, Sci-Fi, Action, Parody, Samurai,...
                               ...                        
12289                                        {Hentai, OVA}
12290                                        {Hentai, OVA}
12291                                        {Hentai, OVA}
12292                                        {Hentai, OVA}
12293                                      {Hentai, Movie}
Name: set, Length: 12294, dtype: object

In [31]:
df["name"] = df["name"].str.lower()
df["name"]=df["name"].str.strip()


In [32]:
df.head()

,anime_id,name,genre,type,episodes,rating,members,set
0,32281,kimi no na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"{Drama, School, Romance, Movie, Supernatural}"
1,5114,fullmetal alchemist: brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"{Adventure, TV, Shounen, Drama, Action, Milita..."
2,28977,gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"{TV, Shounen, Sci-Fi, Action, Parody, Samurai,..."
3,9253,steins;gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"{Sci-Fi, TV, Thriller}"
4,9969,gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"{TV, Shounen, Sci-Fi, Action, Parody, Samurai,..."


In [58]:
df['name']

0                                           kimi no na wa.
1                         fullmetal alchemist: brotherhood
2                                                 gintama°
3                                              steins;gate
4                                            gintama&#039;
                               ...                        
12289         toushindai my lover: minami tai mecha-minami
12290                                          under world
12291                       violence gekiga david no hoshi
12292    violence gekiga shin david no hoshi: inma dens...
12293                     yasuji no pornorama: yacchimae!!
Name: name, Length: 12294, dtype: object

In [33]:
def calculate_score(row,inputset):
    intersection=len(row['set'].intersection(inputset))
    union = len(row['set'].union(inputset))
    return intersection/union

In [39]:
name='Fullmetal Alchemist: Brotherhood'.lower().strip()
inputset=df.loc[df['name'] == name, 'set'].iloc[0]
temp=df[df['name']!=name]
temp['score']=temp.apply(lambda row: calculate_score(row, inputset), axis=1)
temp=temp.sort_values(by='score', ascending=False)
top_5_rows = temp.iloc[:5, :]
top_5_names = top_5_rows['name'].tolist()
top_5_names

['fullmetal alchemist',
 'magi: the labyrinth of magic',
 'densetsu no yuusha no densetsu',
 'magi: sinbad no bouken (tv)',
 'jikuu tenshou nazca']

In [54]:
def Find_recommendation(name):
    if name in df['name'].tolist():
        inputset=df.loc[df['name'] == name, 'set'].iloc[0]
        temp=df[df['name']!=name]
        temp['score']=temp.apply(lambda row: calculate_score(row, inputset), axis=1)
        temp=temp.sort_values(by='score', ascending=False)
        top_5_rows = temp.iloc[:5, :]
        top_5_recommendation = top_5_rows['name'].tolist()
        return top_5_recommendation
    matches = process.extract(name, df['name'].tolist(), limit=1)
    if matches and matches[0][1] >= 80:  
        similar_name = matches[0][0]
        return f"Did you mean '{similar_name}'?"
    return f"'{name}' does not exist in the dataset."

In [62]:
v="fruits basket"
Find_recommendation(v)

['oh! family',
 'hatenkou yuugi',
 'ao haru ride',
 'kareshi kanojo no jijou',
 'suki na mono wa suki dakara shou ga nai!!']

In [59]:
df.to_csv('data.csv',index=False)

In [63]:
import pickle

In [64]:
pickle.dump(df,open('sample.pkl','wb'))

In [67]:
pickle.load(open('sample.pkl','rb'))

,anime_id,name,genre,type,episodes,rating,members,set
0,32281,kimi no na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"{Drama, School, Romance, Movie, Supernatural}"
1,5114,fullmetal alchemist: brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"{Adventure, TV, Shounen, Drama, Action, Milita..."
2,28977,gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"{TV, Shounen, Sci-Fi, Action, Parody, Samurai,..."
3,9253,steins;gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"{Sci-Fi, TV, Thriller}"
4,9969,gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"{TV, Shounen, Sci-Fi, Action, Parody, Samurai,..."
...,...,...,...,...,...,...,...,...
12289,9316,toushindai my lover: minami tai mecha-minami,Hentai,OVA,1,4.15,211,"{Hentai, OVA}"
12290,5543,under world,Hentai,OVA,1,4.28,183,"{Hentai, OVA}"
12291,5621,violence gekiga david no hoshi,Hentai,OVA,4,4.88,219,"{Hentai, OVA}"
12292,6133,violence gekiga shin david no hoshi: inma dens...,Hentai,OVA,1,4.98,175,"{Hentai, OVA}"


In [68]:
dd = pd.read_csv('data.csv')

In [70]:
dd.head()

,anime_id,name,genre,type,episodes,rating,members,set
0,32281,kimi no na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"{'Drama', 'School', 'Romance', 'Movie', 'Super..."
1,5114,fullmetal alchemist: brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"{'Adventure', 'TV', 'Shounen', 'Drama', 'Actio..."
2,28977,gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"{'TV', 'Shounen', 'Sci-Fi', 'Action', 'Parody'..."
3,9253,steins;gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"{'Sci-Fi', 'TV', 'Thriller'}"
4,9969,gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"{'TV', 'Shounen', 'Sci-Fi', 'Action', 'Parody'..."
